In [11]:
from typing import Dict
import warnings
import pandas as pd
import gspread as gs 
import calendar as cd
import streamlit as st
import plotly.graph_objects as go
warnings.simplefilter(action='ignore', category=pd.core.common.SettingWithCopyWarning)


def fetch_data():
    print("""
    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    """)
    gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
    ss = gc.open_by_key(st.secrets['tricount'].spreadsheet_key)
    data = pd.DataFrame(ss.worksheet('Data').get_all_records())
    dict = pd.DataFrame(ss.worksheet('Dict').get_all_records())
    data.to_csv('.out/data.csv')
    dict.to_csv('.out/dict.csv')
    data = pd.read_csv('.out/data.csv')
    dict = pd.read_csv('.out/dict.csv')
    dict = dict[['Postes', 'Catégories']].set_index('Catégories').to_dict()['Postes']
    print(" -- data fetched")
    print(data.shape)
    print(data.columns)
    print(" -- dict2 fetched ")
    print(dict)
    return data, dict


def build_data(data: pd.DataFrame):
    print("""
    # --------------------------------- #
    # Filter, clean and aggregate data  #
    # --------------------------------- #
    """)
    data = data[data['Catégorie'] != ""]
    data['Date'] = pd.to_datetime(data['Date & heure'], format='%d/%m/%Y %H:%M')
    data['Année'] = data['Date'].dt.year
    data['Mois'] = data['Date'].dt.month
    data['Jour'] = data['Date'].dt.day
    data['Lucie'] = data['Impacté à Lucie']
    data['Vincent'] = data['Impacté à Vincent']

    print(data.describe)
    data = data.groupby(['Année', 'Mois', 'Catégorie']).agg({'Lucie': "sum", 'Vincent': "sum"})
    data['Total'] = data['Lucie'] + data['Vincent']
    print(data.describe)

    print(" -- data built")
    print(data.shape)
    print(data.describe)
    print(data)
    return data[['Total', 'Lucie', 'Vincent']]


def format_data(data: pd.DataFrame, dict: Dict):
    print("""
    # --------------------------------- #
    # Format and split data             #
    # by date and categories            #
    # --------------------------------- #
    """)
    tables = {}
    years = data.index.get_level_values('Année').unique().to_list()
    for year in sorted(years, reverse=True):
        df = data.loc[year,:,:]
        months = df.index.get_level_values('Mois').unique().tolist()
        tables[f'{year} (sum)'] = df.groupby('Catégorie').sum()
        tables[f'{year} (mean)'] = tables[f'{year} (sum)'] / len(months)
        for month in sorted(months, reverse=True):
            col_name = f'{year} {cd.month_name[month]}'
            tables[col_name] = df.loc[month,:].groupby('Catégorie').sum()
        # for category in set(dict.values()):
        #     if category not in tables[col_name].columns:
        #         tables[col_name][col_name] = 0
    return tables


In [12]:

data, dict = fetch_data()
dframe = build_data(data)
tables = format_data(dframe)
# result = concat_data(tables, dict)


    # --------------------------------- #
    # Connection to Google Sheet        #
    # and extract Spreadsheets          #
    #  - exported tricount data         #
    #  - category dictonnary            #
    # --------------------------------- #
    
 -- data fetched
(1023, 17)
Index(['Unnamed: 0', 'Titre', 'Montant', 'Devise', 'Taux de change',
       'Montant dans la devise du tricount (EUR)', 'Type de transaction',
       'Catégorie', 'Payé par', 'Payé par Commun', 'Payé par Lucie',
       'Payé par Vincent', 'Impacté à Commun', 'Impacté à Lucie',
       'Impacté à Vincent', 'Date & heure', 'URL des images'],
      dtype='object')
 -- dict2 fetched 
{'Alimentation': 'Quotidien', 'Shopping': 'Achats', 'Logement': 'Achats', 'Loyer & charges': 'Quotidien', 'Divertissement': 'Extra', 'Sport': 'Loisir', 'Restaurant & bar': 'Extra', 'Investissement': 'Investissement', 'Transport': 'Quotidien', 'Week-end': 'Loisir', 'Soin de santé': 'Quotidien', 'Livres': 'Achats', 'Activités': 'Loi

TypeError: format_data() missing 1 required positional argument: 'dict'

In [8]:
dict['Alimentation']

'Quotidien'

In [89]:
result = {}
for table in tables.keys():
    df = tables[table].reset_index()
    df['Poste'] = df['Catégorie'].apply(lambda category: dict[category])
    df = df.set_index('Poste')[['Total', 'Lucie', 'Vincent']]
    df = df.groupby('Poste').sum()
    df = df.transpose()

    for poste in dict['Postes'].unique().tolist():
        if poste not in df.columns:
            df[poste] = 0

    df['Revenus'] = df['Rentrée d\'argent']
    df['Dépenses'] = df['Quotidien'] + df['Loisir'] + df['Extra'] + df['Achats']
    df['Reste à vivre'] = df['Dépenses'] - df['Revenus'] 
    df['Reste à vivre %'] = df['Reste à vivre'] / df['Revenus'] * 100
    df['Capital investi'] = df['Investissement']
    df['Capital investi %'] = df['Capital investi'] / df['Revenus'] * 100
    df['Epargne'] = df['Reste à vivre'] - df['Capital investi']
    df['Epargne %'] = df['Epargne'] / df['Revenus'] * 100
    df = df.transpose()
    print(df)
    result[table] = df



# result

                     Total    Lucie  Vincent
Poste                                       
Achats              345.00   -92.50   437.50
Extra               -81.00   -40.50   -40.50
Investissement      106.00     0.00   106.00
Loisir              -81.02   -40.51   -40.51
Quotidien         -1243.98  -846.14  -397.84
Rentrée d'argent      0.00     0.00     0.00
Revenus               0.00     0.00     0.00
Dépenses          -1061.00 -1019.65   -41.35
Reste à vivre     -1061.00 -1019.65   -41.35
Reste à vivre %       -inf     -inf     -inf
Capital investi     106.00     0.00   106.00
Capital investi %      inf      NaN      inf
Epargne           -1167.00 -1019.65  -147.35
Epargne %             -inf     -inf     -inf
                     Total    Lucie  Vincent
Poste                                       
Achats              345.00   -92.50   437.50
Extra               -81.00   -40.50   -40.50
Investissement      106.00     0.00   106.00
Loisir              -81.02   -40.51   -40.51
Quotidien 